<a href="https://colab.research.google.com/github/liuyao12/imagenette_experiments/blob/master/ResNet_twist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet with a Twist

> with depthwise (x4) + Ranger + Mish + SA + MaxBlurPool + ResTrick

See blog https://liuyao12.github.io/blog/research/2020/03/07/Conv-Twist.html

See summary at https://forums.fast.ai/t/imagenette-imagewoof-leaderboards/45822/47?u=liuyao 

## Imagewoof Leaderboard

Imagewoof2, with a 70/30 train/test ratio.

| Size (px) | Epochs | SoTA| x2 | x4 | x4 twist | x6 | x4 double | runs |
|--|--|--| --|--| --|--|--|--|
|128|5|73.37|75.19|76.27||76.61| **82.12**|5, mean
|128|20|85.52|85.18|86.22||86.27| **88.93**|5, mean
|128|80|87.20|87.70|87.83||87.65| **90.15**|1
|128|200|87.20|
|192|5|77.87|79.86|81.15|80.73|| **82.69**|5, mean
|192|20|87.85|88.12|88.37|88.28|
|192|80|89.21|90.30|90.25|89.38|90.37| **92.08** |
|192|200|89.54
|256|5|
|256|20|
|256|80|
|256|200|


# setup and imports

In [ ]:
# pip install kornia

In [ ]:
pip install git+https://github.com/ayasyrev/model_constructor

In [ ]:
pip install git+https://github.com/ayasyrev/imagenette_experiments

In [3]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *

In [4]:
from kornia.contrib import MaxBlurPool2d

In [5]:
from imagenette_experiments.train_utils import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# ResBlock

In [6]:
class MnM(nn.Module): # Mix and Multiply
    def __init__(self, channels, group_size):
        super().__init__()
        self.channels = channels
        self.gs = group_size
        n = channels//group_size*2
        self.conv = nn.Conv2d(channels, n, 1, groups=1, bias=True)
        self.XY = None

    def forward(self, x): 
        N,C,H,W = x.size()
        # x1 = x.view(N,-1,self.gs,H,W)[:,:,:-2].reshape(N,-1,H,W)
        # x2 = x.view(N,-1,self.gs,H,W)[:,:,:2].reshape(N,-1,H,W)
        if self.XY is None:
            XX = torch.from_numpy(np.indices((1,H,W))[2]*2/W-1)
            YY = torch.from_numpy(np.indices((1,H,W))[1]*2/H-1)
            g = self.channels//self.gs
            self.XY = torch.cat([XX,YY]*g, dim=0).to(x.device).type(x.dtype)
        twist = self.conv(x) * self.XY
        # twist = torch.sum(twist.view(N,-1,2,2,H,W), dim=2).reshape(N,-1,H,W)
        return torch.cat([x, twist], dim=1)

In [13]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ks==3 and groups==1:  # to be used for the "stem" of ResNet
          # if ni==3: stride = 2
          layers = [('Conv3x3', nn.Conv2d(ni, ni*dm, 3, stride=stride, padding=1, bias=bias, groups=ni)),
                    ('Conv1x1', nn.Conv2d(ni*dm, nf, 1, bias=bias, groups=1))]
        else:
          layers = [('Conv{}x{}'.format(ks,ks), 
                      nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [12]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, zero_bn=True, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, groups=1):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(ni, nf, 3, zero_bn=zero_bn, act=False, bn_layer=True))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   # (f"conv_1", conv_layer(nh, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh*dm, 3, groups=nh, act_fn=act_fn, bn_1st=bn_1st)),
                   # (f"conv_1", conv_layer(nh, nh*dm, 3, groups=nh, act=False, bn_layer=False)),
                   # (f"MnM", MnM(nf, dm)),
                   (f"conv_2", conv_layer(nh*dm, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [14]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True

## Experiment

In [15]:
dm = 4
res = dict()
for ep in [80]: #*5 + [20] + [80]:
    mixup=0 if ep<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(ep, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    acc = learn.recorder.metrics[-1][0].item()
    res[ep] = res[ep] + [acc] if ep in res else [acc]
    print('{} epochs: {} ({} runs)'.format(ep, sum(res[ep])/len(res[ep]), len(res[ep])))
print('depth multiplier={}'.format(dm), {ep: sum(res[ep])/len(res[ep]) for ep in res})

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.984969,1.837100,0.413591,0.868669,02:43
1,1.763518,1.476673,0.565284,0.937389,02:42
2,1.602325,1.277940,0.680835,0.961313,02:41
3,1.518453,1.175933,0.721558,0.962077,02:41
4,1.427599,1.112495,0.762026,0.970730,02:40
5,1.373465,1.101031,0.767116,0.973021,02:41
6,1.335170,1.047050,0.789514,0.976839,02:40
7,1.289991,0.991489,0.811911,0.976075,02:40
8,1.273567,0.984097,0.812420,0.977857,02:40
9,1.229387,0.953064,0.831509,0.980657,02:41


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/imagenette_experiments/train_utils.py:150: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


80 epochs: 0.902519702911377 (1 runs)
depth multiplier=4 {80: 0.902519702911377}


In [19]:
dm = 6
for ep in [80]: #*5 + [20] + [80]:
    mixup=0 if ep<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(ep, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    acc = learn.recorder.metrics[-1][0].item()
    res[ep] = res[ep] + [acc] if ep in res else [acc]
    print('{} epochs: {} ({} runs)'.format(ep, sum(res[ep])/len(res[ep]), len(res[ep])))
print('depth multiplier={}'.format(dm), {ep: sum(res[ep])/len(res[ep]) for ep in res})

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.020703,1.802997,0.427335,0.867651,03:09
1,1.779585,1.501510,0.564775,0.925681,03:09
2,1.654090,1.303978,0.665564,0.956732,03:09
3,1.513084,1.190918,0.718503,0.963858,03:09
4,1.462979,1.105817,0.762026,0.970476,03:09
5,1.379337,1.063350,0.775515,0.973530,03:09
6,1.323550,1.003739,0.801731,0.972767,03:09
7,1.273628,0.989483,0.810130,0.980657,03:09
8,1.264744,1.045115,0.777806,0.977348,03:08
9,1.220185,0.947007,0.821583,0.978621,03:09


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


80 epochs: 0.9008229374885559 (3 runs)
depth multiplier=6 {80: 0.9008229374885559}


In [17]:
res

{80: [0.902519702911377, 0.8961567878723145]}

In [18]:
for i in range(80):
  print(i, learn.recorder.val_losses[i].item(), learn.recorder.metrics[i][0].item())

0 1.8019211292266846 0.40519216656684875
1 1.5199214220046997 0.5612115263938904
2 1.334391713142395 0.6543650031089783
3 1.2439961433410645 0.6948332786560059
4 1.1527239084243774 0.7291931509971619
5 1.1206214427947998 0.7495545744895935
6 1.052816390991211 0.7869687080383301
7 1.0666406154632568 0.7770425081253052
8 0.9904943704605103 0.8096207976341248
9 1.0099091529846191 0.7938406467437744
10 0.9543590545654297 0.824128270149231
11 0.956939160823822 0.8218376040458679
12 0.9327539205551147 0.8266734480857849
13 0.9148764610290527 0.8327818512916565
14 0.8986201882362366 0.8434716463088989
15 0.9222990274429321 0.8376176953315735
16 0.9147737622261047 0.8355816006660461
17 0.9077925682067871 0.8429625630378723
18 0.9136068820953369 0.8317638039588928
19 0.8858543038368225 0.8600152730941772
20 0.8733745813369751 0.8539068698883057
21 0.8703872561454773 0.8551794290542603
22 0.9229952692985535 0.8401628732681274
23 0.8630534410476685 0.8577246069908142
24 0.8858904838562012 0.85670